In [67]:
import pydicom
from pydicom.dataset import Dataset
from pydicom.sequence import Sequence
import requests
from pydicom.filebase import DicomBytesIO

# Create some temporary filenames
ds = Dataset()
ds.SpecificCharacterSet = 'ISO_IR 100'
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.88.33'
ds.StudyDate = '20230814'
ds.SeriesTime = '123456.789'
ds.AccessionNumber = ''
ds.Modality = 'SR'
ds.SOPInstanceUID = pydicom.uid.generate_uid()
ds.StudyInstanceUID = '1.400.20.81.610.0002598612943543'  # Study UID
ds.PatientID = '00025986'
ds.SeriesInstanceUID = pydicom.uid.generate_uid()  # Series UID
ds.SeriesDescription = 'AI Report'

#Create a Container Content Item for the Report:
container = Dataset()
container.ValueType = 'CONTAINER'
container.ContinuityOfContent = 'SEPARATE'
container.ConceptNameCodeSequence = Sequence([Dataset()])
container.ConceptNameCodeSequence[0].CodeValue = '121071'
container.ConceptNameCodeSequence[0].CodingSchemeDesignator = 'DCM'
container.ConceptNameCodeSequence[0].CodeMeaning = 'Imaging Measurement Report'
ds.ContentSequence = Sequence([container])

# Create the SCOORD content item
scoord_content = Dataset()
scoord_content.is_little_endian = True
scoord_content.is_implicit_VR = False
scoord_content.RelationshipType = 'CONTAINS'
scoord_content.ValueType = 'SCOORD'
scoord_content.GraphicType = 'POLYGON'
points = [[10, 10], [12, 23], [15, 30], [20, 20]]  # Predicted polygon points
scoord_content.GraphicData = [coord for point in points for coord in point]

# Add a reference to the image
image_reference = Dataset()
image_reference.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'  # SOP Class UID for the image
image_reference.ReferencedSOPInstanceUID = '1.2.3.4.5.6.7.8.11'  # SOP Instance UID for the image
scoord_content.ReferencedSOPSequence = Sequence([image_reference])

# Add the SCOORD content item to the container
container.ContentSequence = Sequence([scoord_content])

file_meta = pydicom.Dataset()
file_meta.FileMetaInformationGroupLength = 196  # Adjust as needed
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.88.33'
file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'  # Explicit VR Little Endian
file_meta.ImplementationClassUID = pydicom.uid.generate_uid()


ds.file_meta = file_meta
ds.is_little_endian = True
ds.is_implicit_VR = False


dicom_bytes_io = DicomBytesIO()
# Write the 128-byte preamble
dicom_bytes_io.write(b"\0" * 128)
# Write the "DICM" signature
dicom_bytes_io.write(b"DICM")
# Write with write_like_original=True to include the DICOM preamble and "DICM" signature
pydicom.filewriter.dcmwrite(dicom_bytes_io, ds, write_like_original=True)
dicom_bytes = dicom_bytes_io.getvalue()


url = 'http://192.168.0.19:8042/instances'  # Update with your Orthanc server's URL
headers = {'Content-Type': 'application/dicom'}
response = requests.post(url, data=dicom_bytes, headers=headers)

if response.status_code == 200:
    print("DICOM file successfully sent to Orthanc server")
else:
    print("Failed to send DICOM file:", response.status_code, response.text)


DICOM file successfully sent to Orthanc server


c:\Users\User\.conda\envs\tf\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR UI: '1.400.20.81.610.0002598612943543'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


In [5]:
from pathlib import Path

import highdicom as hd
import numpy as np
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes
from pydicom.uid import generate_uid
from highdicom.sr.content import FindingSite
from highdicom.sr.templates import Measurement, TrackingIdentifier
from pydicom.valuerep import PersonName
from highdicom.sr import ContainerContentItem
# Path to single-frame CT image instance stored as PS3.10 file
image_file = Path('cspine.dcm')

# Read CT Image data set from PS3.10 files on disk
image_dataset = dcmread(str(image_file))


########################
# Create the root container
root_container = ContainerContentItem(
    name=CodedConcept(value='126000', meaning='Imaging Measurement Report', scheme_designator='DCM'),
    is_content_continuous=False
)

# Create the Language of Content Item and Descendants code content item
language_item = CodeContentItem(
    name=CodedConcept('121049', 'DCM', 'Language of Content Item and Descendants'),
    value=CodedConcept('eng', 'RFC5646', 'English'),
    relationship_type='HAS CONCEPT MOD'
)

# Create the Country of Language code content item
country_item = CodeContentItem(
    name=CodedConcept('121046', 'DCM', 'Country of Language'),
    value=CodedConcept('US', 'ISO3166_1', 'United States'),
    relationship_type='HAS CONCEPT MOD'
)

# Add the country_item to the language_item's ContentSequence
language_item.ContentSequence = [country_item]

# Add the language_item to the root container's ContentSequence
root_container.ContentSequence = [language_item]



##########################
observer_name = PersonName.from_named_components(family_name='Foo', given_name='Bar')
# Describe the context of reported observations: the person that reported
# the observations and the device that was used to make the observations
observer_person_context = hd.sr.ObserverContext(
    observer_type=codes.DCM.Person,
    observer_identifying_attributes=hd.sr.PersonObserverIdentifyingAttributes(
        name=''
    )
)
observer_device_context = hd.sr.ObserverContext(
    observer_type=codes.DCM.Device,
    observer_identifying_attributes=hd.sr.DeviceObserverIdentifyingAttributes(
        uid=hd.UID()
    )
)
observation_context = hd.sr.ObservationContext(
    observer_person_context=observer_person_context,
    observer_device_context=observer_device_context,
)

# Describe the image region for which observations were made
# (in physical space based on the frame of reference)
circle_data = np.array(
    [
        [10.0, 10.0],
        [11.0, 11.0],
    ]
)
circle_item = hd.sr.ScoordContentItem(
    name=codes.DCM.ImageRegion,
    graphic_type=hd.sr.GraphicTypeValues.CIRCLE,
    graphic_data=circle_data,
)

# Describe the anatomic site at which observations were made
finding_sites = [
    FindingSite(
        anatomic_location=codes.SCT.CervicoThoracicSpine,
        topographical_modifier=codes.SCT.VertebralForamen
    ),
]

# Describe the imaging measurements for the image region defined above
measurements = [
    Measurement(
        name=codes.SCT.AreaOfDefinedRegion,
        tracking_identifier=hd.sr.TrackingIdentifier(uid=generate_uid()),
        value=1.7,
        unit=codes.UCUM.SquareMillimeter,
        properties=hd.sr.MeasurementProperties(
            normality=hd.sr.CodedConcept(
                value="17621005",
                meaning="Normal",
                scheme_designator="SCT"
            ),
            level_of_significance=codes.SCT.NotSignificant
        )
    )
]

# The source image from which the measurement was inferred
source_item = hd.sr.CompositeContentItem(
    name=codes.DCM.SourceImage,
    referenced_sop_class_uid=image_dataset.SOPClassUID,
    referenced_sop_instance_uid=image_dataset.SOPInstanceUID,
    relationship_type=hd.sr.RelationshipTypeValues.INFERRED_FROM,
)

# A tracking identifier identifying the measurement
tracking_item = hd.sr.UIDRefContentItem(
    name=codes.DCM.TrackingIdentifier,
    value=hd.UID(),  # a newly generated UID
    relationship_type=hd.sr.RelationshipTypeValues.HAS_OBS_CONTEXT,
)

# Nest the source item below the depth item
circle_item.ContentSequence = [source_item, tracking_item]

referenced_region = hd.sr.ImageRegion(
    graphic_type=hd.sr.GraphicTypeValues.POLYLINE,
    graphic_data=np.array([
        (165.0, 200.0),
        (170.0, 220.0),
        (165.0, 220.0),
        (165.0, 200.0),
    ]),
    source_image=hd.sr.SourceImageForRegion(
        referenced_sop_class_uid=image_dataset.SOPClassUID,
        referenced_sop_instance_uid=image_dataset.SOPInstanceUID
    )
)

planar_roi_measurements = hd.sr.PlanarROIMeasurementsAndQualitativeEvaluations(
    tracking_identifier=TrackingIdentifier(
        uid=hd.UID(),
        identifier='Planar ROI Measurements'
    ),
    finding_type=codes.SCT.SpinalCord,
    referenced_region=referenced_region, 
    measurements=measurements,
    finding_sites=finding_sites
)
planar_roi_measurements.ContentSequence = [root_container]

# Add the PlanarROIMeasurementsAndQualitativeEvaluations object to the imaging_measurements
imaging_measurements = [planar_roi_measurements]

# Create a Container Content Item for the Report
container = ContainerContentItem(
    name=hd.sr.CodedConcept(value='126200', meaning='Imaging Measurement Report', scheme_designator='DCM'),
    is_content_continuous=False
)


# Create the report content
measurement_report = hd.sr.MeasurementReport(
    observation_context=observation_context,
    procedure_reported=codes.LN.CTUnspecifiedBodyRegion,
    imaging_measurements=imaging_measurements
)

# Create the Structured Report instance
sr_dataset = hd.sr.ComprehensiveSR(
    evidence=[image_dataset],
    content=measurement_report,
    series_number=1,
    series_instance_uid=hd.UID(),
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Manufacturer'
)

# dicom_bytes_io = DicomBytesIO()
# # Write with write_like_original=True to include the DICOM preamble and "DICM" signature
# pydicom.filewriter.dcmwrite(dicom_bytes_io, sr_dataset, write_like_original=True)
# dicom_bytes = dicom_bytes_io.getvalue()
# url = 'http://192.168.0.19:8042/instances'  # Update with your Orthanc server's URL
# headers = {'Content-Type': 'application/dicom'}
# response = requests.post(url, data=dicom_bytes, headers=headers)

# if response.status_code == 200:
#     print("DICOM file successfully sent to Orthanc server")
# else:
#     print("Failed to send DICOM file:", response.status_code, response.text)


c:\Users\User\.conda\envs\tf\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR UI: '1.400.20.81.610.0002598612943543'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
c:\Users\User\.conda\envs\tf\lib\site-packages\highdicom\valuerep.py:63: UserWarning: The string "±èÁø±¹" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  UserWarning


In [1]:
from highdicom.sr.value_types import ImageContentItem

def get_image_library(referenced_sop_class_uid,referenced_sop_instance_uid ):
    # Create Image Content
    referenced_image = ImageContentItem(
        referenced_sop_class_uid= referenced_sop_class_uid,
        referenced_sop_instance_uid = referenced_sop_instance_uid,
        relationship_type=RelationshipTypeValues.CONTAINS,
        name=CodedConcept(
            value="121139",
            meaning="Image",
            scheme_designator="DCM",
        )
    )

    # Create Image Library Group Container
    image_library_group = ContainerContentItem(
        name=CodedConcept(
            value='126200',
            scheme_designator='DCM',
            meaning='Image Library Group'
        ),
        is_content_continuous=False,
        relationship_type=RelationshipTypeValues.CONTAINS
    )
    image_library_group.ContentSequence = Sequence([referenced_image])

    # Create Image Library Container
    image_library = ContainerContentItem(
        name=CodedConcept(
            value='111028',
            scheme_designator='DCM',
            meaning='Image Library'
        ),
        is_content_continuous=False,
        relationship_type=RelationshipTypeValues.CONTAINS
    )
    image_library.ContentSequence = Sequence([image_library_group])

    return image_library

In [2]:
points = [[[849, 1491], [849, 1494], [848, 1495], [839, 1495], [839, 1498], [838, 1499], [836, 1499], [836, 1502], [835, 1503], [830, 1503], [830, 1506], [829, 1507], [823, 1507], [823, 1510], [822, 1511], [817, 1511], [817, 1514], [816, 1515], [810, 1515], [810, 1518], [809, 1519], [803, 1519], [803, 1522], [802, 1523], [797, 1523], [797, 1526], [796, 1527], [790, 1527], [790, 1530], [789, 1531], [780, 1531], [780, 1534], [779, 1535], [774, 1535], [774, 1538], [773, 1539], [764, 1539], [764, 1541], [763, 1542], [748, 1542], [748, 1545], [747, 1546], [744, 1546], [744, 1549], [743, 1550], [741, 1550], [741, 1557], [743, 1557], [744, 1558], [744, 1569], [747, 1569], [748, 1570], [748, 1577], [750, 1577], [751, 1578], [751, 1585], [753, 1585], [754, 1586], [754, 1601], [757, 1601], [758, 1602], [758, 1609], [760, 1609], [761, 1610], [761, 1625], [763, 1625], [764, 1626], [764, 1640], [766, 1640], [767, 1641], [767, 1648], [770, 1648], [771, 1649], [771, 1660], [773, 1660], [774, 1661], [774, 1664], [786, 1664], [786, 1661], [787, 1660], [793, 1660], [793, 1657], [794, 1656], [799, 1656], [799, 1653], [800, 1652], [802, 1652], [802, 1649], [803, 1648], [809, 1648], [809, 1645], [810, 1644], [812, 1644], [812, 1641], [813, 1640], [819, 1640], [819, 1637], [820, 1636], [825, 1636], [825, 1633], [826, 1632], [832, 1632], [832, 1630], [833, 1629], [838, 1629], [838, 1626], [839, 1625], [848, 1625], [848, 1622], [849, 1621], [868, 1621], [868, 1618], [869, 1617], [884, 1617], [884, 1614], [885, 1613], [891, 1613], [891, 1606], [892, 1605], [894, 1605], [894, 1602], [892, 1602], [891, 1601], [891, 1594], [889, 1594], [888, 1593], [888, 1586], [885, 1586], [884, 1585], [884, 1578], [882, 1578], [881, 1577], [881, 1570], [879, 1570], [878, 1569], [878, 1558], [876, 1558], [875, 1557], [875, 1550], [872, 1550], [871, 1549], [871, 1535], [869, 1535], [868, 1534], [868, 1519], [866, 1519], [865, 1518], [865, 1503], [862, 1503], [861, 1502], [861, 1495], [859, 1495], [858, 1494], [858, 1491]], [[813, 1353], [813, 1356], [812, 1357], [807, 1357], [807, 1360], [806, 1361], [797, 1361], [797, 1364], [796, 1365], [787, 1365], [787, 1367], [786, 1368], [774, 1368], [774, 1371], [773, 1372], [764, 1372], [764, 1375], [763, 1376], [751, 1376], [751, 1379], [750, 1380], [741, 1380], [741, 1383], [740, 1384], [731, 1384], [731, 1387], [730, 1388], [725, 1388], [725, 1391], [724, 1392], [718, 1392], [718, 1395], [717, 1396], [715, 1396], [715, 1399], [714, 1400], [712, 1400], [712, 1423], [714, 1423], [715, 1424], [715, 1431], [717, 1431], [718, 1432], [718, 1447], [720, 1447], [721, 1448], [721, 1486], [720, 1487], [718, 1487], [718, 1506], [734, 1506], [734, 1503], [735, 1502], [740, 1502], [740, 1499], [741, 1498], [747, 1498], [747, 1495], [748, 1494], [750, 1494], [750, 1491], [751, 1490], [757, 1490], [757, 1487], [758, 1486], [763, 1486], [763, 1483], [764, 1482], [773, 1482], [773, 1479], [774, 1478], [783, 1478], [783, 1475], [784, 1474], [793, 1474], [793, 1471], [794, 1470], [802, 1470], [802, 1467], [803, 1466], [848, 1466], [848, 1452], [846, 1452], [845, 1451], [845, 1444], [843, 1444], [842, 1443], [842, 1436], [839, 1436], [838, 1435], [838, 1424], [836, 1424], [835, 1423], [835, 1416], [833, 1416], [832, 1415], [832, 1404], [830, 1404], [829, 1403], [829, 1380], [826, 1380], [825, 1379], [825, 1353]], [[800, 1206], [800, 1209], [799, 1210], [794, 1210], [794, 1213], [793, 1214], [787, 1214], [787, 1217], [786, 1218], [780, 1218], [780, 1221], [779, 1222], [774, 1222], [774, 1225], [773, 1226], [761, 1226], [761, 1229], [760, 1230], [741, 1230], [741, 1233], [740, 1234], [725, 1234], [725, 1237], [724, 1238], [715, 1238], [715, 1241], [714, 1242], [708, 1242], [708, 1245], [707, 1246], [702, 1246], [702, 1249], [701, 1250], [695, 1250], [695, 1253], [694, 1254], [692, 1254], [692, 1265], [691, 1266], [689, 1266], [689, 1269], [691, 1269], [692, 1270], [692, 1280], [694, 1280], [695, 1281], [695, 1296], [697, 1296], [698, 1297], [698, 1328], [697, 1329], [695, 1329], [695, 1360], [701, 1360], [702, 1361], [702, 1364], [707, 1364], [707, 1361], [708, 1360], [714, 1360], [714, 1357], [715, 1356], [717, 1356], [717, 1353], [718, 1352], [724, 1352], [724, 1349], [725, 1348], [730, 1348], [730, 1345], [731, 1344], [734, 1344], [734, 1341], [735, 1340], [743, 1340], [743, 1337], [744, 1336], [753, 1336], [753, 1333], [754, 1332], [763, 1332], [763, 1329], [764, 1328], [779, 1328], [779, 1325], [780, 1324], [799, 1324], [800, 1325], [800, 1328], [809, 1328], [810, 1329], [810, 1332], [819, 1332], [819, 1329], [820, 1328], [822, 1328], [822, 1313], [820, 1313], [819, 1312], [819, 1297], [817, 1297], [816, 1296], [816, 1285], [813, 1285], [812, 1284], [812, 1222], [810, 1222], [809, 1221], [809, 1214], [807, 1214], [806, 1213], [806, 1206]], [[767, 1072], [767, 1075], [766, 1076], [758, 1076], [758, 1079], [757, 1080], [744, 1080], [744, 1083], [743, 1084], [738, 1084], [738, 1087], [737, 1088], [725, 1088], [725, 1091], [724, 1092], [712, 1092], [712, 1095], [711, 1096], [702, 1096], [702, 1099], [701, 1100], [685, 1100], [685, 1102], [684, 1103], [679, 1103], [679, 1106], [678, 1107], [672, 1107], [672, 1114], [671, 1115], [669, 1115], [669, 1134], [671, 1134], [672, 1135], [672, 1166], [675, 1166], [676, 1167], [676, 1209], [678, 1209], [679, 1210], [679, 1217], [681, 1217], [682, 1218], [682, 1221], [684, 1221], [684, 1218], [685, 1217], [691, 1217], [691, 1214], [692, 1213], [697, 1213], [697, 1210], [698, 1209], [701, 1209], [701, 1206], [702, 1205], [707, 1205], [707, 1202], [708, 1201], [717, 1201], [717, 1198], [718, 1197], [727, 1197], [727, 1194], [728, 1193], [734, 1193], [734, 1190], [735, 1189], [747, 1189], [747, 1187], [748, 1186], [793, 1186], [794, 1187], [794, 1189], [802, 1189], [802, 1187], [803, 1186], [806, 1186], [806, 1175], [803, 1175], [802, 1174], [802, 1159], [800, 1159], [799, 1158], [799, 1151], [797, 1151], [796, 1150], [796, 1135], [794, 1135], [793, 1134], [793, 1107], [790, 1107], [789, 1106], [789, 1076], [787, 1076], [786, 1075], [786, 1072]], [[761, 914], [761, 917], [760, 918], [754, 918], [754, 921], [753, 922], [748, 922], [748, 924], [747, 925], [741, 925], [741, 928], [740, 929], [735, 929], [735, 932], [734, 933], [725, 933], [725, 936], [724, 937], [715, 937], [715, 940], [714, 941], [702, 941], [702, 944], [701, 945], [689, 945], [689, 948], [688, 949], [679, 949], [679, 952], [678, 953], [672, 953], [672, 956], [671, 957], [669, 957], [669, 964], [668, 965], [666, 965], [666, 968], [665, 969], [662, 969], [662, 1004], [661, 1005], [659, 1005], [659, 1035], [661, 1035], [662, 1036], [662, 1047], [661, 1048], [659, 1048], [659, 1059], [661, 1059], [662, 1060], [662, 1071], [665, 1071], [666, 1072], [666, 1075], [678, 1075], [678, 1072], [679, 1071], [681, 1071], [681, 1068], [682, 1067], [688, 1067], [688, 1064], [689, 1063], [691, 1063], [691, 1060], [692, 1059], [701, 1059], [701, 1056], [702, 1055], [707, 1055], [707, 1052], [708, 1051], [714, 1051], [714, 1048], [715, 1047], [720, 1047], [720, 1044], [721, 1043], [730, 1043], [730, 1040], [731, 1039], [766, 1039], [767, 1040], [767, 1043], [783, 1043], [783, 1040], [784, 1039], [786, 1039], [786, 1024], [784, 1024], [783, 1023], [783, 1009], [780, 1009], [779, 1008], [779, 985], [777, 985], [776, 984], [776, 941], [774, 941], [773, 940], [773, 922], [771, 922], [770, 921], [770, 918], [767, 918], [766, 917], [766, 914]], [[954, 664], [954, 667], [953, 668], [948, 668], [948, 671], [947, 672], [944, 672], [944, 675], [943, 676], [938, 676], [938, 679], [937, 680], [935, 680], [935, 683], [934, 684], [931, 684], [931, 687], [930, 688], [928, 688], [928, 691], [927, 692], [925, 692], [925, 695], [924, 696], [921, 696], [921, 703], [920, 704], [918, 704], [918, 707], [917, 708], [915, 708], [915, 711], [914, 712], [912, 712], [912, 723], [911, 724], [908, 724], [908, 739], [911, 739], [912, 740], [912, 743], [920, 743], [921, 744], [921, 747], [937, 747], [938, 748], [938, 750], [953, 750], [953, 748], [954, 747], [957, 747], [957, 744], [958, 743], [960, 743], [960, 740], [961, 739], [963, 739], [963, 732], [964, 731], [966, 731], [966, 696], [964, 696], [963, 695], [963, 676], [961, 676], [960, 675], [960, 668], [958, 668], [957, 667], [957, 664]], [[702, 597], [702, 600], [701, 601], [695, 601], [695, 604], [694, 605], [692, 605], [692, 608], [691, 609], [689, 609], [689, 612], [688, 613], [685, 613], [685, 616], [684, 617], [682, 617], [682, 624], [681, 625], [679, 625], [679, 636], [678, 637], [676, 637], [676, 648], [675, 649], [672, 649], [672, 667], [671, 668], [669, 668], [669, 691], [668, 692], [666, 692], [666, 699], [665, 700], [662, 700], [662, 715], [665, 715], [666, 716], [666, 747], [665, 748], [662, 748], [662, 754], [661, 755], [659, 755], [659, 762], [658, 763], [656, 763], [656, 802], [658, 802], [659, 803], [659, 822], [658, 823], [656, 823], [656, 837], [655, 838], [653, 838], [653, 857], [652, 858], [649, 858], [649, 877], [652, 877], [653, 878], [653, 921], [655, 921], [656, 922], [656, 928], [671, 928], [671, 925], [672, 924], [678, 924], [678, 922], [679, 921], [684, 921], [684, 918], [685, 917], [691, 917], [691, 914], [692, 913], [697, 913], [697, 910], [698, 909], [704, 909], [704, 906], [705, 905], [711, 905], [711, 902], [712, 901], [724, 901], [724, 898], [725, 897], [734, 897], [734, 894], [735, 893], [773, 893], [773, 886], [771, 886], [770, 885], [770, 878], [767, 878], [766, 877], [766, 866], [767, 865], [770, 865], [770, 862], [771, 861], [779, 861], [780, 862], [780, 865], [786, 865], [787, 866], [787, 869], [793, 869], [794, 870], [794, 873], [796, 873], [797, 874], [797, 877], [802, 877], [803, 878], [803, 881], [806, 881], [807, 882], [807, 889], [809, 889], [810, 890], [810, 893], [816, 893], [817, 894], [817, 901], [819, 901], [820, 902], [820, 905], [822, 905], [823, 906], [823, 909], [825, 909], [826, 910], [826, 913], [829, 913], [830, 914], [830, 917], [832, 917], [833, 918], [833, 921], [838, 921], [839, 922], [839, 924], [865, 924], [866, 925], [866, 928], [868, 928], [868, 925], [869, 924], [878, 924], [878, 922], [879, 921], [888, 921], [888, 918], [889, 917], [907, 917], [907, 914], [908, 913], [943, 913], [944, 914], [944, 917], [957, 917], [958, 918], [958, 921], [966, 921], [967, 922], [967, 924], [973, 924], [974, 925], [974, 928], [983, 928], [984, 929], [984, 932], [989, 932], [990, 933], [990, 936], [993, 936], [994, 937], [994, 940], [996, 940], [997, 941], [997, 944], [1002, 944], [1003, 945], [1003, 948], [1006, 948], [1007, 949], [1007, 952], [1009, 952], [1010, 953], [1010, 960], [1012, 960], [1013, 961], [1013, 968], [1016, 968], [1017, 969], [1017, 972], [1022, 972], [1023, 973], [1023, 976], [1035, 976], [1035, 973], [1036, 972], [1042, 972], [1042, 969], [1043, 968], [1045, 968], [1045, 953], [1046, 952], [1048, 952], [1048, 902], [1046, 902], [1045, 901], [1045, 886], [1043, 886], [1042, 885], [1042, 874], [1040, 874], [1039, 873], [1039, 866], [1036, 866], [1035, 865], [1035, 858], [1033, 858], [1032, 857], [1032, 854], [1030, 854], [1029, 853], [1029, 846], [1026, 846], [1025, 845], [1025, 842], [1023, 842], [1022, 841], [1022, 838], [1020, 838], [1019, 837], [1019, 835], [1017, 835], [1016, 834], [1016, 831], [1013, 831], [1012, 830], [1012, 827], [1010, 827], [1009, 826], [1009, 823], [1007, 823], [1006, 822], [1006, 819], [1000, 819], [999, 818], [999, 815], [994, 815], [993, 814], [993, 811], [980, 811], [979, 810], [979, 807], [971, 807], [970, 806], [970, 803], [944, 803], [943, 802], [943, 799], [908, 799], [908, 802], [907, 803], [905, 803], [904, 802], [904, 799], [862, 799], [861, 798], [861, 795], [836, 795], [835, 794], [835, 791], [820, 791], [819, 790], [819, 787], [813, 787], [812, 786], [812, 783], [807, 783], [806, 782], [806, 779], [797, 779], [796, 778], [796, 775], [790, 775], [789, 774], [789, 771], [784, 771], [783, 770], [783, 767], [777, 767], [776, 766], [776, 763], [774, 763], [773, 762], [773, 759], [767, 759], [766, 758], [766, 755], [761, 755], [760, 754], [760, 751], [754, 751], [753, 750], [753, 748], [751, 748], [750, 747], [750, 740], [748, 740], [747, 739], [747, 736], [744, 736], [743, 735], [743, 732], [741, 732], [740, 731], [740, 724], [741, 723], [743, 723], [743, 716], [741, 716], [740, 715], [740, 708], [738, 708], [737, 707], [737, 696], [738, 695], [740, 695], [740, 676], [741, 675], [743, 675], [743, 637], [741, 637], [740, 636], [740, 633], [738, 633], [737, 632], [737, 625], [735, 625], [734, 624], [734, 621], [731, 621], [730, 620], [730, 613], [725, 613], [724, 612], [724, 609], [721, 609], [720, 608], [720, 605], [718, 605], [717, 604], [717, 601], [708, 601], [707, 600], [707, 597]], [[659, 581], [659, 584], [658, 585], [653, 585], [653, 588], [652, 589], [649, 589], [649, 592], [648, 593], [643, 593], [643, 596], [642, 597], [639, 597], [639, 600], [638, 601], [636, 601], [636, 604], [635, 605], [633, 605], [633, 608], [632, 609], [630, 609], [630, 612], [629, 613], [626, 613], [626, 620], [625, 621], [623, 621], [623, 628], [622, 629], [620, 629], [620, 632], [622, 632], [623, 633], [623, 652], [625, 652], [626, 653], [626, 660], [629, 660], [630, 661], [630, 663], [632, 663], [633, 664], [633, 667], [645, 667], [646, 668], [646, 671], [655, 671], [655, 664], [656, 663], [658, 663], [658, 653], [659, 652], [661, 652], [661, 633], [662, 632], [665, 632], [665, 621], [666, 620], [668, 620], [668, 613], [669, 612], [671, 612], [671, 605], [672, 604], [675, 604], [675, 593], [672, 593], [671, 592], [671, 589], [669, 589], [668, 588], [668, 585], [666, 585], [665, 584], [665, 581]], [[1148, 467], [1148, 470], [1147, 471], [1141, 471], [1141, 474], [1140, 475], [1135, 475], [1135, 478], [1134, 479], [1128, 479], [1128, 482], [1127, 483], [1125, 483], [1125, 485], [1124, 486], [1118, 486], [1118, 489], [1117, 490], [1115, 490], [1115, 493], [1114, 494], [1112, 494], [1112, 497], [1111, 498], [1105, 498], [1105, 501], [1104, 502], [1099, 502], [1099, 505], [1098, 506], [1092, 506], [1092, 509], [1091, 510], [1085, 510], [1085, 513], [1084, 514], [1079, 514], [1079, 517], [1078, 518], [1072, 518], [1072, 521], [1071, 522], [1066, 522], [1066, 525], [1065, 526], [1059, 526], [1059, 529], [1058, 530], [1053, 530], [1053, 533], [1052, 534], [1046, 534], [1046, 537], [1045, 538], [1040, 538], [1040, 541], [1039, 542], [1033, 542], [1033, 545], [1032, 546], [1030, 546], [1030, 549], [1029, 550], [1026, 550], [1026, 553], [1025, 554], [1023, 554], [1023, 557], [1022, 558], [1020, 558], [1020, 561], [1019, 562], [1017, 562], [1017, 569], [1016, 570], [1013, 570], [1013, 576], [1012, 577], [1010, 577], [1010, 580], [1009, 581], [1007, 581], [1007, 584], [1006, 585], [1003, 585], [1003, 588], [1002, 589], [1000, 589], [1000, 604], [1019, 604], [1019, 601], [1020, 600], [1029, 600], [1029, 597], [1030, 596], [1039, 596], [1039, 593], [1040, 592], [1058, 592], [1058, 589], [1059, 588], [1071, 588], [1071, 585], [1072, 584], [1084, 584], [1084, 581], [1085, 580], [1098, 580], [1098, 577], [1099, 576], [1111, 576], [1111, 573], [1112, 572], [1121, 572], [1121, 570], [1122, 569], [1134, 569], [1134, 566], [1135, 565], [1143, 565], [1143, 562], [1144, 561], [1153, 561], [1153, 558], [1154, 557], [1160, 557], [1160, 554], [1161, 553], [1166, 553], [1166, 550], [1167, 549], [1173, 549], [1173, 546], [1174, 545], [1180, 545], [1180, 542], [1181, 541], [1189, 541], [1189, 538], [1190, 537], [1196, 537], [1196, 534], [1197, 533], [1206, 533], [1206, 530], [1207, 529], [1209, 529], [1209, 526], [1210, 525], [1216, 525], [1216, 522], [1217, 521], [1219, 521], [1219, 514], [1217, 514], [1216, 513], [1216, 510], [1213, 510], [1212, 509], [1212, 506], [1210, 506], [1209, 505], [1209, 502], [1203, 502], [1202, 501], [1202, 498], [1197, 498], [1196, 497], [1196, 494], [1190, 494], [1189, 493], [1189, 490], [1187, 490], [1186, 489], [1186, 486], [1184, 486], [1183, 485], [1183, 483], [1181, 483], [1180, 482], [1180, 479], [1177, 479], [1176, 478], [1176, 475], [1174, 475], [1173, 474], [1173, 471], [1171, 471], [1170, 470], [1170, 467]], [[148, 435], [148, 438], [147, 439], [121, 439], [121, 458], [124, 458], [125, 459], [125, 466], [127, 466], [128, 467], [128, 470], [196, 470], [197, 471], [197, 474], [212, 474], [213, 475], [213, 478], [235, 478], [236, 479], [236, 482], [242, 482], [242, 479], [243, 478], [261, 478], [261, 475], [262, 474], [278, 474], [278, 471], [279, 470], [291, 470], [292, 471], [292, 474], [304, 474], [305, 475], [305, 478], [317, 478], [318, 479], [318, 482], [360, 482], [360, 475], [351, 475], [350, 474], [350, 471], [334, 471], [333, 470], [333, 467], [321, 467], [320, 466], [320, 463], [305, 463], [304, 462], [304, 459], [292, 459], [291, 458], [291, 455], [282, 455], [281, 454], [281, 451], [269, 451], [268, 450], [268, 447], [256, 447], [255, 446], [255, 443], [243, 443], [242, 442], [242, 439], [220, 439], [219, 438], [219, 435]], [[636, 293], [636, 296], [635, 297], [630, 297], [630, 300], [629, 301], [623, 301], [623, 304], [622, 305], [620, 305], [620, 307], [619, 308], [610, 308], [610, 311], [609, 312], [600, 312], [600, 315], [599, 316], [590, 316], [590, 319], [589, 320], [584, 320], [584, 323], [583, 324], [574, 324], [574, 327], [573, 328], [564, 328], [564, 331], [563, 332], [554, 332], [554, 335], [553, 336], [548, 336], [548, 339], [550, 339], [551, 340], [551, 343], [556, 343], [557, 344], [557, 347], [563, 347], [564, 348], [564, 351], [570, 351], [571, 352], [571, 355], [576, 355], [577, 356], [577, 359], [579, 359], [580, 360], [580, 363], [586, 363], [587, 364], [587, 367], [589, 367], [590, 368], [590, 371], [596, 371], [597, 372], [597, 375], [599, 375], [600, 376], [600, 379], [602, 379], [603, 380], [603, 383], [606, 383], [607, 384], [607, 387], [609, 387], [610, 388], [610, 391], [612, 391], [613, 392], [613, 395], [616, 395], [617, 396], [617, 398], [619, 398], [620, 399], [620, 402], [622, 402], [623, 403], [623, 406], [625, 406], [626, 407], [626, 410], [629, 410], [630, 411], [630, 414], [632, 414], [633, 415], [633, 418], [635, 418], [636, 419], [636, 422], [638, 422], [639, 423], [639, 426], [642, 426], [643, 427], [643, 430], [645, 430], [646, 431], [646, 434], [648, 434], [649, 435], [649, 438], [652, 438], [653, 439], [653, 442], [658, 442], [659, 443], [659, 446], [665, 446], [666, 447], [666, 450], [668, 450], [669, 451], [669, 454], [675, 454], [676, 455], [676, 458], [678, 458], [679, 459], [679, 466], [681, 466], [682, 467], [682, 470], [684, 470], [685, 471], [685, 478], [688, 478], [689, 479], [689, 482], [691, 482], [692, 483], [692, 489], [694, 489], [695, 490], [695, 501], [697, 501], [698, 502], [698, 509], [701, 509], [702, 510], [702, 521], [704, 521], [705, 522], [705, 525], [707, 525], [707, 522], [708, 521], [714, 521], [714, 518], [715, 517], [717, 517], [717, 514], [718, 513], [720, 513], [720, 483], [718, 483], [717, 482], [717, 467], [715, 467], [714, 466], [714, 455], [712, 455], [711, 454], [711, 447], [708, 447], [707, 446], [707, 439], [705, 439], [704, 438], [704, 427], [702, 427], [701, 426], [701, 419], [698, 419], [697, 418], [697, 411], [695, 411], [694, 410], [694, 403], [692, 403], [691, 402], [691, 399], [689, 399], [688, 398], [688, 392], [685, 392], [684, 391], [684, 388], [682, 388], [681, 387], [681, 380], [679, 380], [678, 379], [678, 368], [676, 368], [675, 367], [675, 356], [672, 356], [671, 355], [671, 348], [669, 348], [668, 347], [668, 340], [666, 340], [665, 339], [665, 332], [662, 332], [661, 331], [661, 320], [659, 320], [658, 319], [658, 308], [656, 308], [655, 307], [655, 305], [653, 305], [652, 304], [652, 297], [649, 297], [648, 296], [648, 293]]]


In [3]:
from highdicom.sr.value_types import (
    ContainerContentItem,
    UIDRefContentItem, 
    NumContentItem,
    TextContentItem, 
    ScoordContentItem, 
    ImageContentItem)
from highdicom.sr.coding import CodedConcept
from highdicom.sr import RelationshipTypeValues
from pydicom.sequence import Sequence
from pydicom.filereader import dcmread
def create_scoord_item(sop_class_uid, sop_instance_uid, data):
    graphic_data = data
    # Create a SCOORD content item for the graphic data
    scoord_item = ScoordContentItem (
        graphic_type=GraphicTypeValues.POLYLINE,
        graphic_data=graphic_data,
        relationship_type=RelationshipTypeValues.INFERRED_FROM,
        name = CodedConcept(value='G-A166', meaning='Scoord', scheme_designator='SRT')  # Modify as needed
        )
    
    # Create an IMAGE content item for the referenced SOP
    referenced_image = ImageContentItem(
        referenced_sop_class_uid= sop_class_uid, # '1.2.840.10008.5.1.4.1.1.1.1',
        referenced_sop_instance_uid=sop_instance_uid, #'1.3.46.670589.30.1.6.1.1625523293.1622603492578.2', #sop_instance_uid,
        relationship_type=RelationshipTypeValues.SELECTED_FROM,
        name = CodedConcept(value='G-A166', meaning='Scoord', scheme_designator='SRT')  # Modify as needed
    )
    scoord_item.ContentSequence = Sequence([referenced_image])
    return scoord_item

# Scoord not supported in OHIF
def create_measurement(scoord_item):
    # Create a NUM content item for the area measurement
    area_measurement = NumContentItem(
        name=CodedConcept(value='G-A166', meaning='', scheme_designator='SRT'),
        value=0,
        unit=CodedConcept(value='', meaning='SquareMilliMeter', scheme_designator='UCUM'),
        relationship_type=RelationshipTypeValues.CONTAINS
    )
    area_measurement.ContentSequence = Sequence([scoord_item])
    return area_measurement

def create_imaging_measurement_container(sop_class_uid, sop_instance_uid, data):

    
    traking_identifier = TextContentItem(
        relationship_type=RelationshipTypeValues.HAS_OBS_CONTEXT,
        name=CodedConcept(value='112039', meaning='Tracking Identifier', scheme_designator='DCM'),
        value= 'Cornerstone3DTools@^0.1.0:PlanarFreehandROI'
    )
    
    tracking_uid = UIDRefContentItem(
        relationship_type=RelationshipTypeValues.HAS_OBS_CONTEXT,
        name=CodedConcept(value='112040', meaning='Tracking Unique Identifier', scheme_designator='DCM'),
        value=hd.UID()
    )
    # Create a container for the measurement group
    measurement_group = ContainerContentItem(
        name=CodedConcept(value='125007', meaning='Measurement Group', scheme_designator='DCM'),
        is_content_continuous=False,
        relationship_type=RelationshipTypeValues.CONTAINS
    )
    
    measurement_group.ContentSequence = Sequence([traking_identifier, tracking_uid]) #area_measurement,
    
    # add polygons to measurement group
    # polygon = np.array(data[0])
    # scoord_item = create_scoord_item(sop_class_uid, sop_instance_uid, polygon)
    # measurement = create_measurement(scoord_item)
    # measurement_group.ContentSequence.append(measurement)
    for i in data:
        i.append(i[0]) # closed shape
        polygon = np.array(i)
        scoord_item = create_scoord_item(sop_class_uid, sop_instance_uid, polygon)
        measurement = create_measurement(scoord_item)
        measurement_group.ContentSequence.append(measurement)

    # Create the root container for imaging measurements
    imaging_measurements = ContainerContentItem(
        name=CodedConcept(value='126010', meaning='Imaging Measurements', scheme_designator='DCM'),
        is_content_continuous=False,
        relationship_type=RelationshipTypeValues.CONTAINS
    )
    imaging_measurements.ContentSequence = Sequence([measurement_group])

    return imaging_measurements



In [5]:
from highdicom.sr import (
    PlanarROIMeasurementsAndQualitativeEvaluations,
    ImageRegion,
    SourceImageForRegion,
    FindingSite,
    Measurement,
    TrackingIdentifier,
    RelationshipTypeValues,
    ContentSequence,
    MeasurementReport,
    GraphicTypeValues,
)
from pydicom.sr.codedict import codes
from highdicom.sr import ComprehensiveSR
from highdicom.sr.coding import CodedConcept
from highdicom.sr.value_types import CodeContentItem, ContainerContentItem
import highdicom as hd
from pathlib import Path
import numpy as np
# Path to single-frame CT image instance stored as PS3.10 file
image_file = Path('cspine.dcm')

# Read CT Image data set from PS3.10 files on disk
image_dataset = dcmread(str(image_file))


# Create the root container
root_container = ContainerContentItem(
    name=CodedConcept(value='126000', meaning='Imaging Measurement Report', scheme_designator='DCM'),
    is_content_continuous=False
)

# Create the Language of Content Item and Descendants code content item
language_item = CodeContentItem(
    name=CodedConcept('121049', 'DCM', 'Language of Content Item and Descendants'),
    value=CodedConcept('eng', 'RFC5646', 'English'),
    relationship_type='HAS CONCEPT MOD'
)

# Create the Country of Language code content item
country_item = CodeContentItem(
    name=CodedConcept('121046', 'DCM', 'Country of Language'),
    value=CodedConcept('US', 'ISO3166_1', 'United States'),
    relationship_type='HAS CONCEPT MOD'
)

# Add the country_item to the language_item's ContentSequence
language_item.ContentSequence = [country_item]

image_library_container = get_image_library(image_dataset.SeriesInstanceUID, image_dataset.SOPInstanceUID)

# Add the language_item to the root container's ContentSequence
root_container.ContentSequence = [language_item, image_library_container]

# measurement_container = create_imaging_measurement_container(image_dataset.SOPClassUID, image_dataset.SOPInstanceUID,data)
# root_container.ContentSequence.append(measurement_container)


measurement_container = create_imaging_measurement_container(image_dataset.SOPClassUID, image_dataset.SOPInstanceUID,points)
root_container.ContentSequence.append(measurement_container)



# Add the language_item to the root container's ContentSequence


#print(root_container)

In [9]:
from pydicom.filebase import DicomBytesIO
import pydicom
import requests
def send_to_server(sr_dataset): 
    dicom_bytes_io = DicomBytesIO()
    # Write with write_like_original=True to include the DICOM preamble and "DICM" signature
    pydicom.filewriter.dcmwrite(dicom_bytes_io, sr_dataset, write_like_original=True)
    dicom_bytes = dicom_bytes_io.getvalue()
    print(len(dicom_bytes))
    url = 'http://192.168.0.19/orthanc/instances'  # Update with your Orthanc server's URL
    headers = {'Content-Type': 'application/dicom'}
    response = requests.post(url, data=dicom_bytes, headers=headers)

    if response.status_code == 200:
        print("DICOM file successfully sent to Orthanc server")
    else:
        print("Failed to send DICOM file:", response.status_code, response.text)

In [10]:
# Create the Structured Report instance
sr_dataset = hd.sr.ComprehensiveSR(
    evidence=[image_dataset],
    content=root_container,
    series_number=1,
    series_instance_uid=hd.UID(),
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Manufacturer',
    is_complete = True,
    series_description ="AI",
   
    
)


send_to_server(sr_dataset)

21956
DICOM file successfully sent to Orthanc server


c:\Users\User\.conda\envs\tf\lib\site-packages\highdicom\valuerep.py:63: UserWarning: The string "±èÁø±¹" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  UserWarning
c:\Users\User\.conda\envs\tf\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR UI: '1.400.20.81.610.0002598612943543'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


In [220]:

# The source image from which the measurement was inferred
source_item = hd.sr.CompositeContentItem(
    name=codes.DCM.SourceImage,
    referenced_sop_class_uid=image_dataset.SOPClassUID,
    referenced_sop_instance_uid=image_dataset.SOPInstanceUID,
    relationship_type=hd.sr.RelationshipTypeValues.INFERRED_FROM,
)

# A tracking identifier identifying the measurement
tracking_item = hd.sr.UIDRefContentItem(
    name=codes.DCM.TrackingIdentifier,
    value=hd.UID(),  # a newly generated UID
    relationship_type=hd.sr.RelationshipTypeValues.HAS_OBS_CONTEXT,
)

# Nest the source item below the depth item
circle_item.ContentSequence = [source_item, tracking_item]

referenced_region = hd.sr.ImageRegion(
    graphic_type=hd.sr.GraphicTypeValues.POLYLINE,
    graphic_data=np.array([
        (165.0, 200.0),
        (170.0, 220.0),
        (165.0, 220.0),
        (165.0, 200.0),
    ]),
    source_image=hd.sr.SourceImageForRegion(
        referenced_sop_class_uid=image_dataset.SOPClassUID,
        referenced_sop_instance_uid=image_dataset.SOPInstanceUID
    )
)

planar_roi_measurements = hd.sr.PlanarROIMeasurementsAndQualitativeEvaluations(
    tracking_identifier=TrackingIdentifier(
        uid=hd.UID(),
        identifier='Planar ROI Measurements'
    ),
    finding_type=codes.SCT.SpinalCord,
    referenced_region=referenced_region, 
    measurements=measurements,
    finding_sites=finding_sites
)
planar_roi_measurements.ContentSequence = [circle_item]


# Add the nested container to the root container's content sequence
root_container.ContentSequence.append(circle_item)

print(root_container.ContentSequence)
# Create the report content
measurement_report = hd.sr.MeasurementReport(
    observation_context=observation_context,
    procedure_reported=codes.LN.CTUnspecifiedBodyRegion,
    imaging_measurements=[planar_roi_measurements]
)

# Create the Structured Report instance
sr_dataset = hd.sr.ComprehensiveSR(
    evidence=[image_dataset],
    content=measurement_report,
    series_number=1,
    series_instance_uid=hd.UID(),
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Manufacturer'
)

print(sr_dataset)

AttributeError: Items to be appended to a ContentSequence must have an established relationship type.

In [173]:


# The source image from which the measurement was inferred
source_item = hd.sr.CompositeContentItem(
    name=codes.DCM.SourceImage,
    referenced_sop_class_uid=image_dataset.SOPClassUID,
    referenced_sop_instance_uid=image_dataset.SOPInstanceUID,
    relationship_type=hd.sr.RelationshipTypeValues.INFERRED_FROM,
)

# A tracking identifier identifying the measurement
tracking_item = hd.sr.UIDRefContentItem(
    name=codes.DCM.TrackingIdentifier,
    value=hd.UID(),  # a newly generated UID
    relationship_type=hd.sr.RelationshipTypeValues.HAS_OBS_CONTEXT,
)

# Nest the source item below the depth item
circle_item.ContentSequence = [source_item, tracking_item]

referenced_region = hd.sr.ImageRegion(
    graphic_type=hd.sr.GraphicTypeValues.POLYLINE,
    graphic_data=np.array([
        (165.0, 200.0),
        (170.0, 220.0),
        (165.0, 220.0),
        (165.0, 200.0),
    ]),
    source_image=hd.sr.SourceImageForRegion(
        referenced_sop_class_uid=image_dataset.SOPClassUID,
        referenced_sop_instance_uid=image_dataset.SOPInstanceUID
    )
)

planar_roi_measurements = hd.sr.PlanarROIMeasurementsAndQualitativeEvaluations(
    tracking_identifier=TrackingIdentifier(
        uid=hd.UID(),
        identifier='Planar ROI Measurements'
    ),
    finding_type=codes.SCT.SpinalCord,
    referenced_region=referenced_region, 
    measurements=measurements,
    finding_sites=finding_sites
)
planar_roi_measurements.ContentSequence = [circle_item]



# Add the PlanarROIMeasurementsAndQualitativeEvaluations object to the nested container's content sequence
nested_container.ContentSequence = [planar_roi_measurements]

# Add the nested container to the root container's content sequence
root_container.ContentSequence.append(nested_container)

# Create the report content
measurement_report = hd.sr.MeasurementReport(
    observation_context=observation_context,
    procedure_reported=codes.LN.CTUnspecifiedBodyRegion,
    imaging_measurements=root_container
)

# Create the Structured Report instance
sr_dataset = hd.sr.ComprehensiveSR(
    evidence=[image_dataset],
    content=measurement_report,
    series_number=1,
    series_instance_uid=hd.UID(),
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Manufacturer'
)



TypeError: Sequence contents must be Dataset instances.

In [174]:
# Describe the image region for the ROI
referenced_region = ImageRegion(
    graphic_type='POLYLINE',  # or another type depending on your ROI
    graphic_data=np.array([[10.0, 10.0], [50.0, 50.0]]),  # coordinates of the ROI
    source_image=SourceImageForRegion(
        referenced_sop_class_uid=image_dataset.SOPClassUID,
        referenced_sop_instance_uid=image_dataset.SOPInstanceUID
    )
)

# Describe the anatomic site (optional)
finding_sites = [
    FindingSite(
        anatomic_location=codes.SCT.CervicoThoracicSpine,
        topographical_modifier=codes.SCT.VertebralForamen
    ),
]

# Describe the measurements for the ROI
measurements = [
    Measurement(
        name=codes.SCT.AreaOfDefinedRegion,
        tracking_identifier=TrackingIdentifier(
            uid=hd.UID(),
            identifier='Planar ROI Measurements'
        ),
        value=1.7,  # example value
        unit=codes.UCUM.SquareMillimeter
    )
]
# Create the PlanarROIMeasurementsAndQualitativeEvaluations object
planar_roi_measurements = PlanarROIMeasurementsAndQualitativeEvaluations(
    tracking_identifier=TrackingIdentifier(
        uid=hd.UID(),
        identifier='Planar ROI Measurements'
    ),
    finding_type=codes.SCT.SpinalCord,
    referenced_region=referenced_region,
    measurements=measurements,
    finding_sites=finding_sites
)



# Add the PlanarROIMeasurementsAndQualitativeEvaluations object to the nested container's content sequence
nested_container.ContentSequence.append(planar_roi_measurements)

# Add the nested container to the root container's content sequence
root_container.ContentSequence.append(nested_container)
# Create the MeasurementReport instance
measurement_report = MeasurementReport(
    observation_context=observation_context,
    procedure_reported=codes.LN.CTUnspecifiedBodyRegion,
    imaging_measurements=[root_container]
)
# Create the SR instance with the root container as the main content
sr_dataset = ComprehensiveSR(
    evidence=[image_dataset], # Add your image dataset here
    content=measurement_report,
    series_number=1,
    series_instance_uid='Your Series Instance UID',
    sop_instance_uid='Your SOP Instance UID',
    instance_number=1,
    manufacturer='Manufacturer'
)


TypeError: Items of "ContentSequence" must have type ContentItem.

Create a DICOM Dataset and Add Basic Information:


In [42]:
ds = Dataset()
ds.SpecificCharacterSet = 'ISO_IR 100'
ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.88.22'  # Enhanced SR
ds.StudyDate = '20230814'
ds.SeriesTime = '123456.789'
ds.AccessionNumber = ''
ds.Modality = 'SR'
ds.SOPInstanceUID = pydicom.uid.generate_uid()
ds.StudyInstanceUID = '1.400.20.81.610.0002598612943543'  # Study UID
ds.SeriesInstanceUID = pydicom.uid.generate_uid()  # Series UID



c:\Users\User\.conda\envs\tf\lib\site-packages\pydicom\valuerep.py:290: UserWarning: Invalid value for VR UI: '1.400.20.81.610.0002598612943543'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


Create a Container Content Item for the Report:


In [43]:
container = Dataset()
container.ValueType = 'CONTAINER'
container.ContinuityOfContent = 'SEPARATE'
container.ConceptNameCodeSequence = Sequence([Dataset()])
container.ConceptNameCodeSequence[0].CodeValue = '121071'
container.ConceptNameCodeSequence[0].CodingSchemeDesignator = 'DCM'
container.ConceptNameCodeSequence[0].CodeMeaning = 'Imaging Measurement Report'
ds.ContentSequence = Sequence([container])


Create a SCOORD Content Item for the Annotation:


In [44]:
# Create the SCOORD content item
scoord_content = Dataset()
scoord_content.is_little_endian = True
scoord_content.is_implicit_VR = False
scoord_content.RelationshipType = 'CONTAINS'
scoord_content.ValueType = 'SCOORD'
scoord_content.GraphicType = 'POLYGON'
points = [[10, 10], [12, 23], [15, 30], [20, 20]]  # Add your points here
scoord_content.GraphicData = [coord for point in points for coord in point]
#scoord_content.save_as('scoord_content.dcm', write_like_original=False)

Associate the SCOORD Content Item with an Image:


In [45]:
# Add a reference to the image
image_reference = Dataset()
image_reference.ReferencedSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'  # SOP Class UID for the image
image_reference.ReferencedSOPInstanceUID = '1.2.3.4.5.6.7.8.11'  # SOP Instance UID for the image
scoord_content.ReferencedSOPSequence = Sequence([image_reference])

# Add the SCOORD content item to the container
container.ContentSequence = Sequence([scoord_content])


Create a FileMetaDataset:


In [46]:
file_meta = pydicom.Dataset()
file_meta.FileMetaInformationGroupLength = 196  # Adjust as needed
file_meta.FileMetaInformationVersion = b'\x00\x01'
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.88.22'  # Enhanced SR
file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'  # Explicit VR Little Endian
file_meta.ImplementationClassUID = pydicom.uid.generate_uid()



In [47]:
ds.file_meta = file_meta
ds.is_little_endian = True
ds.is_implicit_VR = False
ds.save_as('annotation.dcm', write_like_original=False)

In [48]:
from pydicom.filebase import DicomBytesIO
dicom_bytes_io = DicomBytesIO()
pydicom.filewriter.dcmwrite(dicom_bytes_io, ds)
dicom_bytes = dicom_bytes_io.getvalue()

In [49]:
url = 'http://192.168.0.19:8042/instances'  # Update with your Orthanc server's URL
headers = {'Content-Type': 'application/dicom'}
response = requests.post(url, data=dicom_bytes, headers=headers)

if response.status_code == 200:
    print("DICOM file successfully sent to Orthanc server")
else:
    print("Failed to send DICOM file:", response.status_code, response.text)


DICOM file successfully sent to Orthanc server
